In [59]:
class test():
    def __init__(self):
        self.knowledge = []
    
    def add_knowledge(self, count):
        self.knowledge.append(count)

In [56]:
a = [1, 2, 3]
b = 4
a += [b]
print(a)

[1, 2, 3, 4]


In [58]:
a = {1, 2, 3}
b = {2, 3}
a -= b
print(a)

{1}


In [64]:
a = test()
print(a.knowledge)
a.add_knowledge(2)
a.add_knowledge(10)
print(a.knowledge)

[]
[2, 10]


In [66]:
a = set()
for e in a:
    print(a)

In [67]:
a = [1, 1, 2, 2, 3, 4, 5, 5, 5]
print(a)
print(list(set(a)))

[1, 1, 2, 2, 3, 4, 5, 5, 5]
[1, 2, 3, 4, 5]


In [70]:
10 in a

False

In [72]:
a = {1, 2, 3}
b = {3, 2, 1, 1}

a == b

True

In [73]:
a = set()
a

set()

In [74]:
if a:
    print("hello")